In [1]:
import pandas as pd
import numpy as np
import gzip
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.


In [2]:
"""
Source : http://jmcauley.ucsd.edu/data/amazon/qa/ 
"""
def parse(path): 
    g = gzip.open(path, 'rb') 
    for l in g: 
        yield eval(l) 
        
def getDF(path): 
    i = 0 
    df = {} 
    for d in parse(path): 
        df[i] = d 
        i += 1 
    return pd.DataFrame.from_dict(df, orient='index') 

In [3]:
def clean(questions):
    table = str.maketrans('', '', string.punctuation)
    for i,q in enumerate(questions):
        # tokenize
        x = q.split()
        # lower case
        x = [w.lower() for w  in x]
        # remove punctuation
        x = [w.translate(table) for w in x]
        # remove single-letter stuff
        x = [w for w in x if len(w) > 1]
        # remove numbers?
        x = [w for w in x if w.isalpha()]
        
        questions[i] = ' '.join(x)

In [4]:
fname = 'qa_Appliances.json.gz'
# fname = 'qa_Beauty.json.gz'
fname = 'qa_Electronics.json.gz'

In [5]:
df = getDF(fname)[['questionType','question']]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314263 entries, 0 to 314262
Data columns (total 2 columns):
questionType    314263 non-null object
question        314263 non-null object
dtypes: object(2)
memory usage: 7.2+ MB


In [7]:
from collections import Counter
Counter(df.questionType)

Counter({'yes/no': 165598, 'open-ended': 148665})

In [8]:
yn = Counter(df.questionType)['yes/no']
oe = Counter(df.questionType)['open-ended']
p = yn*100/(yn+oe)
print("yes/no : {0:.2f}%\topen-ended : {1:.2f}%".format(p, 100-p))

yes/no : 52.69%	open-ended : 47.31%


In [9]:
type(df.question)

pandas.core.series.Series

In [10]:
questions = df['question'].values.tolist()
# clean(questions)
X = pd.Series(questions)
Y = df.questionType

In [11]:
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [12]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)#, random_state=42)

Processing the data
- Tokenize and convert text to seq
- add padding all seq has same shape
- choose arbitrary max_len. many ways exist

In [13]:
max_words = 1000#20000
max_len = 100#len(max(X))//2
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)

In [14]:
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [15]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [16]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.3)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [17]:
model = RNN()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 50)           50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [18]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 213698 samples, validate on 53425 samples
Epoch 1/10
213698/213698 [==============================] - 283s 1ms/step - loss: 0.2987 - acc: 0.8732 - val_loss: 0.2221 - val_acc: 0.9109
Epoch 2/10
213698/213698 [==============================] - 230s 1ms/step - loss: 0.2059 - acc: 0.9198 - val_loss: 0.1928 - val_acc: 0.9263
Epoch 3/10
213698/213698 [==============================] - 230s 1ms/step - loss: 0.1889 - acc: 0.9299 - val_loss: 0.1827 - val_acc: 0.9321
Epoch 4/10
213698/213698 [==============================] - 236s 1ms/step - loss: 0.1785 - acc: 0.9343 - val_loss: 0.1792 - val_acc: 0.9340
Epoch 5/10
213698/213698 [==============================] - 244s 1ms/step - loss: 0.1735 - acc: 0.9364 - val_loss: 0.1759 - val_acc: 0.9360
Epoch 6/10
213698/213698 [==============================] - 244s 1ms/step - loss: 0.1719 - acc: 0.9366 - val_loss: 0.1748 - val_acc: 0.9367
Epoch 7/10
213698/213698 [==============================] - 235s 1ms/step - loss: 0.1662 - acc: 0.9388 - val_

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

47104/47140 [============================>.] - ETA: 0s